In [61]:
# Paso 1 Cargamos las librerias
import seaborn as sn #Graficas
import pandas as pd #Manejar la data
import matplotlib.pyplot as plt #Graficas
from sklearn.metrics import multilabel_confusion_matrix #Matriz de Confusuion

# Importando Regresion Lineal 
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression # Regression logistica
from sklearn.model_selection import train_test_split, cross_val_score

# Importando Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Graficas y Arbol de Decisiones
import graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score

# Imposte de lobrerias para labels en matris de Confusion
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

import pickle

from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.combine import SMOTETomek

In [62]:
# Cargando la data
data = pd.read_csv("parcial.csv")

In [63]:
# Paso 2 Verificamos si hay valores nulos 
data.dropna(inplace=True)
data.shape

(299, 13)

In [64]:
# Paso 2 Parte 2 - Verificamos datos repetidos
print(f'Tamano del Set antes de eliminar repetidos: {data.shape}')
data.drop_duplicates(inplace=True)
print(f'Tamano del Set despues de eliminar repetidos: {data.shape}')

Tamano del Set antes de eliminar repetidos: (299, 13)
Tamano del Set despues de eliminar repetidos: (299, 13)


In [65]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       299 non-null    float64
 1   anaemia                   299 non-null    int64  
 2   creatinine_phosphokinase  299 non-null    int64  
 3   diabetes                  299 non-null    int64  
 4   ejection_fraction         299 non-null    int64  
 5   high_blood_pressure       299 non-null    int64  
 6   platelets                 299 non-null    float64
 7   serum_creatinine          299 non-null    float64
 8   serum_sodium              299 non-null    int64  
 9   sex                       299 non-null    int64  
 10  smoking                   299 non-null    int64  
 11  time                      299 non-null    int64  
 12  DEATH_EVENT               299 non-null    int64  
dtypes: float64(3), int64(10)
memory usage: 30.5 KB


In [66]:
# Paso 3 Equilibrio de Datos - Target
num_muertos = data['DEATH_EVENT'].sum()
num_noMuertos = 300 - num_muertos
#num_noMuertos = data['DEATH_EVENT']==False
print("Muertos:",num_muertos)
print("No Muertos:",num_noMuertos)

Muertos: 96
No Muertos: 204


In [67]:
# Paso 4 Calcular la correlacion entre variables de time y Death_Event
# ya que el time es el tiempo de seguimiento del tratamiento y el Death 
# Event nos confirma si murió o no, dentro del tiempo de seguimiento
correlation = data["time"].corr(data["DEATH_EVENT"])
print("Correlacion entre el Time y Death Event: ",correlation)

Correlacion entre el Time y Death Event:  -0.526963779277577


In [75]:
# Paso 5 Separar los datos en entrenamiento y prueba 
# Porcentaje y otros detalles en cuenta
y = data['DEATH_EVENT']
X = data.iloc[:, 0:12]

In [76]:
# ROS para Duplicar las muestras de la clase menos representada y 
# Balancear los datos
ros = RandomOverSampler()
dataRos, targetRos=ros.fit_resample(X,y)
muertas = targetRos.sum()
no_muertas = targetRos.shape[0] - muertas
print("Muertas:",muertas," No Muertas:", no_muertas)
dataRos

Muertas: 203  No Muertas: 203


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time
0,75.0,0,582,0,20,1,265000.00,1.90,130,1,0,4
1,55.0,0,7861,0,38,0,263358.03,1.10,136,1,0,6
2,65.0,0,146,0,20,0,162000.00,1.30,129,1,1,7
3,50.0,1,111,0,20,0,210000.00,1.90,137,1,0,7
4,65.0,1,160,1,20,0,327000.00,2.70,116,0,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...
401,86.0,0,582,0,38,0,263358.03,1.83,134,0,0,95
402,50.0,0,582,1,38,0,310000.00,1.90,135,1,1,35
403,65.0,0,113,1,25,0,497000.00,1.83,135,1,0,67
404,70.0,1,125,0,25,1,237000.00,1.00,140,0,0,15


In [79]:
# Paso 5 Entrenamiento y Prueba
# Regresion Logistica 
# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


# Paso 6 Algoritmos segun Target - LogisticRegression, RandomForestClassifier
model = LogisticRegression(max_iter=1000).fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluacion del Modelo
print("Precision del Modelo Regresion Logistica:", model.score(X_test, y_test))
# Imprimir los coeficientes de la regresión
print("Coeficiente de la pendiente:", model.coef_[0])
print("Término independiente:", model.intercept_)

# Random Forest
# Inicializar y entrenar el Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)
# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print("Precisión del modelo en Random Forest:", accuracy)

Precision del Modelo Regresion Logistica: 0.9
Coeficiente de la pendiente: [ 5.34717326e-02 -9.56010223e-04  2.04063190e-04  5.29371763e-03
 -5.90866074e-02 -3.18140899e-03 -2.47001645e-06  3.76280714e-02
  5.60937192e-03 -3.80961414e-03 -2.86707445e-03 -1.83024956e-02]
Término independiente: [0.00082519]
Precision media de CrossValidation: 0.7784574468085106
Precisión del modelo en Random Forest: 0.9166666666666666


In [81]:
# Paso 7 - Cross Validation
# CrossValidation k-1
scores = cross_val_score(model, X_train, y_train, cv=5)
#Calculamos la precision media de CrossValidation
mean_accuracy = scores.mean()
print("Precision media de CrossValidation:", mean_accuracy)

Precision media de CrossValidation: 0.8203014184397162


In [80]:
# Paso 7 Matriz de Confusion
# Matriz de Confusion
# Dividimos la data entre Validacion de Train y Validacion de Test
Xt, Xcv, Yt, Ycv = train_test_split(X_train, y_train, test_size=0.20, random_state=1)
# Hacemos prediccion basado en el modelo del Arbol de Desicion
Y_hat = model.predict(Xcv)
print("Precision de la Validacion de Data Test:", accuracy_score(Ycv, Y_hat))
# Impresion de la amtriz de Confusion para random state de 2
multilabel_confusion_matrix(Ycv, Y_hat)

Precision de la Validacion de Data Test: 1.0


array([[[20,  0],
        [ 0, 28]],

       [[28,  0],
        [ 0, 20]]], dtype=int64)